In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from gensim import corpora
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import os, re, operator, warnings
warnings.filterwarnings('ignore')  

import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

C:\Users\Estefi\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\Estefi\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Estefi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:

def preprocess(text):
    result = []
    #Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [3]:
dreamers=pd.read_csv("../data/dreamers_summary_old.csv",delimiter="|",encoding="latin-1")
print('# de palabras del Corpus: '+ str(dreamers[['total_words']].sum()))
dreamers

# de palabras del Corpus: total_words    7011558.0
dtype: float64


,group,dreamer sex,dreamer age,dream years,numbers of dreams,summary,id,total_words
0,Alta: a detailed dreamer,female,adult,1985-1997,422,Alta is an adult woman who wrote down her drea...,1,166351.0
1,Angie: age 18 & 20,female,18 to 20,1996,48,Angie is a college student whose dreams are of...,2,9754.0
2,Arlie: a middle-aged woman,female,middle-aged,1992-1998,212,Arlie is an older adult woman who wrote down h...,3,22022.0
3,Barb Sanders,female,20 to 60,1960-1997,3116,Barb Sanders (not her real name) is a middle-a...,4,576351.0
4,Barb Sanders #2,female,60 to 64,1997-2001,1138,"1138 more dreams from Barb Sanders, written do...",5,217901.0
...,...,...,...,...,...,...,...,...
84,Vietnam Vet: 1970-2008 war dreams,male,21 to 59,1970-2008,98,There are three sets of dreams from this Vietn...,85,14755.0
85,Vietnam Vet: 2015 dreams,male,66,2015,32,There are three sets of dreams from this Vietn...,86,3618.0
86,Vietnam Vet: 2016-17 dreams,male,67 to 68,2016-2017,463,There are three sets of dreams from this Vietn...,87,62311.0
87,Wedding dreams,female,?,1940s-1950s & 1990s,65,This set consists of college women's dreams th...,88,7132.0


In [4]:
dreamers[dreamers['id'].isin([18,26,27,79,80])] # german groups

,group,dreamer sex,dreamer age,dream years,numbers of dreams,summary,id,total_words
17,"Detlev von Uslar, auf Deutsch",male,adult,1949-2001,6100,"This remarkable series of 6,100 dreams in Germ...",18,835138.0
25,German dreams (F),female,young-adult,1990s,397,These dream reports -- in German -- were colle...,26,41878.0
26,German dreams (M),male,young-adult,1990s,140,These dream reports -- in German -- were colle...,27,15519.0
78,"Swiss children, auf Deutsch (F)",female,9 to 14,1989-1995,164,This collection contains 299 dreams (164 girls...,79,26677.0
79,"Swiss children, auf Deutsch (M)",male,9 to 14,1989-1995,135,This collection contains 299 dreams (164 girls...,80,18171.0


In [5]:
#Removing German Dreamers
print('#dreams: ' + str(dreamers['numbers of dreams'].sum()))
dreamers.drop([17,25,26,78,79], inplace=True)
dreamers.shape
print('#dreams en ingles: ' + str(dreamers['numbers of dreams'].sum()))

#dreams: 43141
#dreams en ingles: 36205


In [6]:
print('# de palabras sin las series en aleman: ' + str(dreamers[['total_words']].sum()))


# de palabras sin las series en aleman: total_words    6074175.0
dtype: float64


In [7]:
cond=dreamers['id'].isin([71,72,73])
phil=dreamers[cond]
print('#Sueños de Phil: ' + str(phil['numbers of dreams'].sum()))

#Sueños de Phil: 506


In [8]:
print(dreamers["dreamer sex"].unique())
df=dreamers.groupby('dreamer sex').agg({'dreamer sex':'count', 'numbers of dreams': 'sum'})
print(df)
df.apply(lambda x:  100*x / x.sum())

['female' 'male']
             dreamer sex  numbers of dreams
dreamer sex                                
female                53              27723
male                  31               8482


,dreamer sex,numbers of dreams
dreamer sex,,
female,63.095238,76.572297
male,36.904762,23.427703


In [9]:
print(dreamers["dreamer age"].unique())
dreamers["dreamer age"].replace(['6 to 8','9 to 11','10','12','12 to 13','13', '7 to 13'],'middle-childhood',inplace=True)
dreamers["dreamer age"].replace(['12 to 14','14 to 16','14','15','16','17','15 to 17','16 to 18','14 to 18','13 to 20','11 to 18'],'teenagers',inplace=True)
dreamers["dreamer age"].replace(['18','18 to 19','19 to 21','18 to 24','23','21 to 22','24','18 to 21','22','18 to 20','21','24 to 25','20','29'],'young-adult',inplace=True)
dreamers["dreamer age"].replace(['62','50','66','67 to 68','60 to 64' ],'middle-aged',inplace=True)
dreamers["dreamer age"].replace(['30','34 to 38'],'adult',inplace=True)
dreamers["dreamer age"].replace(['20 to 60','25 to 75','57 to 80','40 to 80','12 to 21','12 to 25','14 to 27','22 to 43','21 to 59'],'series',inplace=True)

['adult' '18 to 20' 'middle-aged' '20 to 60' '60 to 64' '9 to 11'
 '12 to 14' '14 to 16' '?' '50' 'young-adult' '21' '13 to 20' '25 to 75'
 '57 to 80' '40 to 80' '12 to 21' '12 to 25' '12' '13' '14' '15' '16' '17'
 '18' '22' '14 to 27' '16 to 18' '18 to 21' '24 to 25' '20' '18 to 19'
 '19 to 21' '21 to 22' '6 to 8' '7 to 13' '23' '30' '14 to 18' '34 to 38'
 '15 to 17' '29' '62' '22 to 43' '24' '12 to 13' '18 to 24' '10'
 '21 to 59' '66' '67 to 68' '11 to 18']


In [10]:
print(dreamers["dreamer age"].unique())
dreamers[dreamers['dreamer age']=='?']

['adult' 'young-adult' 'middle-aged' 'series' 'middle-childhood'
 'teenagers' '?']


,group,dreamer sex,dreamer age,dream years,numbers of dreams,summary,id,total_words
10,Blind dreamers (F),female,?,mid-1990s,238,These dreams were collected in the mid-1990s f...,11,35883.0
11,Blind dreamers (M),male,?,mid-1990s,143,These dreams were collected in the mid-1990s f...,12,18701.0
27,Hall/VdC Norms: Female,female,?,1940s-1950s,490,This set contains the 500 dream reports on whi...,28,63757.0
28,Hall/VdC Norms: Male,male,?,1940s-1950s,491,This set contains the 500 dream reports on whi...,29,58325.0
39,"Izzy, uncertain dates",female,?,1997-,23,"This dream series consists of 4,352 dream repo...",40,1412.0
87,Wedding dreams,female,?,1940s-1950s & 1990s,65,This set consists of college women's dreams th...,88,7132.0


In [15]:
dreamers.to_csv("../data/dreamers_summary.csv", sep="|", index=False)

In [11]:
df_etario=dreamers[dreamers['dreamer age'].isin(['adult','young-adult','middle-aged','middle-childhood', 'teenagers'])]
print('#Sueños x rango etario en total: ' + str(df_etario['numbers of dreams'].sum()))

#Sueños x rango etario en total: 18789


In [12]:
df=dreamers.groupby('dreamer age').agg({'dreamer age':'count', 'numbers of dreams': 'sum'})
print(df)
df.apply(lambda x:  100*x / x.sum())

                  dreamer age  numbers of dreams
dreamer age                                     
?                           6               1450
adult                       4               3679
middle-aged                 9               3505
middle-childhood            7                669
series                     13              15966
teenagers                  16               3861
young-adult                29               7075


,dreamer age,numbers of dreams
dreamer age,,
?,7.142857,4.004972
adult,4.761905,10.161580
middle-aged,10.714286,9.680983
middle-childhood,8.333333,1.847811
series,15.476190,44.098881
teenagers,19.047619,10.664273
young-adult,34.523810,19.541500


In [13]:
#Check how preprocess works.
texto=dreamers[['summary']].values[2,0]
texto
print('original document: ')
words = []
for word in texto.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized document: ')
print(preprocess(texto))

original document: 
['Arlie', 'is', 'an', 'older', 'adult', 'woman', 'who', 'wrote', 'down', 'her', 'dreams', 'in', 'the', '1990s.', 'She', 'gave', 'them', 'to', 'us', 'in', '1998.', 'This', 'series', 'has', 'never', 'been', 'studied,', 'except', 'by', 'the', 'dreamer.']


 tokenized document: 
['arlie', 'older', 'adult', 'woman', 'wrote', 'dreams', 'gave', 'series', 'studied', 'dreamer']


In [14]:
documents = dreamers[['summary']]

In [15]:
processed_docs = documents['summary'].map(preprocess)
processed_docs[:10]

0    [alta, adult, woman, wrote, dreams, late, earl...
1    [angie, college, student, dreams, wrote, diffe...
2    [arlie, older, adult, woman, wrote, dreams, ga...
3    [barb, sanders, real, middle, aged, adult, wom...
4    [dreams, barb, sanders, written, dreams, inclu...
5    [dreams, representative, sample, dream, report...
6    [dreams, collected, november, january, februar...
7    [dreams, collected, november, january, februar...
8    [vivid, dream, reports, provide, longest, deta...
9                                                   []
Name: summary, dtype: object

In [16]:
dictionary = Dictionary(processed_docs)
print(len(dictionary))
dictionary.filter_extremes(no_below=10, no_above=0.5)
print(len(dictionary))
corpus = [dictionary.doc2bow(text) for text in processed_docs]

860
38


In [17]:
ldamodel = LdaModel(corpus=corpus, num_topics=25, id2word=dictionary, iterations = 2000, passes=10)
ldamodel.print_topics(10, 6)

[(20,
  '0.448*"early" + 0.209*"life" + 0.158*"college" + 0.081*"young" + 0.003*"recorded" + 0.003*"begins"'),
 (24,
  '0.170*"ages" + 0.168*"years" + 0.113*"dreamer" + 0.086*"including" + 0.086*"theories" + 0.085*"hall"'),
 (23,
  '0.209*"college" + 0.152*"woman" + 0.139*"young" + 0.125*"studied" + 0.110*"unique" + 0.109*"year"'),
 (14,
  '0.093*"reports" + 0.093*"ages" + 0.062*"written" + 0.062*"izzy" + 0.062*"early" + 0.031*"information"'),
 (22,
  '0.178*"year" + 0.178*"journal" + 0.119*"interesting" + 0.119*"theories" + 0.119*"information" + 0.119*"student"'),
 (0,
  '0.194*"young" + 0.194*"information" + 0.099*"years" + 0.099*"wrote" + 0.099*"woman" + 0.099*"available"'),
 (12,
  '0.026*"time" + 0.026*"year" + 0.026*"recorded" + 0.026*"consists" + 0.026*"theories" + 0.026*"journal"'),
 (3,
  '0.236*"hall" + 0.173*"written" + 0.149*"collected" + 0.130*"reports" + 0.066*"young" + 0.066*"studied"'),
 (5,
  '0.256*"available" + 0.220*"information" + 0.146*"time" + 0.110*"studied" + 0

In [18]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.057592  0.013602       1        1  40.442719
17     0.211503 -0.139349       2        1   6.798996
1     -0.041172  0.012375       3        1   6.483172
18     0.027971  0.106133       4        1   5.161162
3      0.057038 -0.115059       5        1   4.833440
23    -0.191689  0.286310       6        1   4.802444
8      0.135617 -0.086432       7        1   3.819817
15     0.168130  0.199648       8        1   3.279831
5     -0.316946 -0.127199       9        1   3.041354
21    -0.041291  0.053939      10        1   2.725390
13     0.008248 -0.107627      11        1   2.556398
4     -0.007659 -0.068727      12        1   2.550983
9     -0.011485 -0.147356      13        1   2.143322
10    -0.178917 -0.180275      14        1   2.015345
16     0.107809  0.130197      15        1   1.824364
20    -0.145808  0.206474      16        1   1.542372
24     0.147192 -0.050957      17        1   1.284879
0     -0.215142 -0.068159      18        1   1.198706
22     0.042716  0.053643      19        1   1.094066
7      0.084901  0.007506      20        1   0.735946
6      0.011536  0.008839      21        1   0.333057
11     0.034818 -0.001823      22        1   0.333057
19     0.031966 -0.003382      23        1   0.333057
2      0.011536  0.008839      24        1   0.333057
12     0.011536  0.008839      25        1   0.333057, topic_info=   Category       Freq         Term      Total  loglift  logprob
4   Default  43.000000         ages  43.000000  30.0000  30.0000
0   Default  37.000000        early  37.000000  29.0000  29.0000
12  Default  44.000000      written  44.000000  28.0000  28.0000
16  Default  60.000000      reports  60.000000  27.0000  27.0000
11  Default  34.000000  information  34.000000  26.0000  26.0000
..      ...        ...          ...        ...      ...      ...
11  Topic25   0.074587  information  34.420212  -0.4298  -3.6376
4   Topic25   0.074587         ages  43.539207  -0.6648  -3.6376
18  Topic25   0.074587    including  32.241322  -0.3644  -3.6376
19  Topic25   0.074587        years  32.925808  -0.3854  -3.6376
20  Topic25   0.074587        young  37.004101  -0.5022  -3.6376

[922 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4         1  0.734970       ages
4         8  0.137807       ages
4        10  0.045936       ages
4        17  0.045936       ages
8         9  0.460427  available
...     ...       ...        ...
20        9  0.081072      young
20       10  0.108096      young
20       11  0.027024      young
20       16  0.027024      young
20       18  0.054048      young

[190 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 18, 2, 19, 4, 24, 9, 16, 6, 22, 14, 5, 10, 11, 17, 21, 25, 1, 23, 8, 7, 12, 20, 3, 13])

In [19]:
ldatopics = [[word for word, prob in topic] for topicid, topic in ldamodel.show_topics(25,formatted=False)]
lda_coherence = CoherenceModel(topics=ldatopics, texts=processed_docs, dictionary=dictionary, window_size=30,coherence='c_npmi',topn=5).get_coherence_per_topic()
df_lda = pd.DataFrame([range(25),lda_coherence,ldatopics],index=['topic_id','coherence','topic']).transpose()
df_lda.sort_values(by='coherence',ascending=False)[:10]

,topic_id,coherence,topic
14,14,0.554338,"[reports, ages, izzy, written, early, informat..."
19,19,0.444361,"[jasmine, reports, written, recorded, includin..."
11,11,0.420636,"[reports, written, ages, including, recorded, ..."
17,17,0.399732,"[written, reports, consists, including, record..."
23,23,0.229632,"[college, woman, young, studied, unique, year,..."
18,18,0.211084,"[year, student, period, college, time, informa..."
1,1,0.195908,"[jasmine, recorded, young, cast, characters, i..."
9,9,0.152516,"[characters, cast, years, information, journal..."
5,5,0.152434,"[available, information, time, studied, young,..."
24,24,0.140443,"[ages, years, dreamer, including, theories, ha..."
